## [ Chapter 13 - Semantic Search with Dense Vectors ] 
## Setting up the outdoors dataset

We're going to use the Outdoors dataset for this chapter, and for a very important reason: the vocabulary and contexts in the outdoor question and answer __domain__ already have good coverage in the Transformer models we'll be using.

This is because the datasets that were used to train the model include sources that are likely to have similar subject matter.  Wikipedia was used to train bert-base-uncased (https://huggingface.co/bert-base-uncased#training-data) and, surprise! wikipedia has a section specifically on outdoors content: https://en.wikipedia.org/wiki/Outdoor

This is important, because if the words and their contexts haven't been seen before, the model will be less accurate.

Also, who doesn't enjoy playing around with a new dataset?! Data is search nerd candy.

In [1]:
import html
import json

from IPython.display import HTML, display
import aips.indexer
from aips import get_engine

engine = get_engine()

## Listing 13.1

### Creating our Collection and Indexing the documents

In [10]:
outdoors_collection = aips.indexer.build_collection(engine, "outdoors")

## Listing 13.2

### Exploring data for a question regarding `climbing knots`

In [3]:
def get_posts_for_question(query):
    request = {"query": query,
               "limit": 1, 
               "query_fields": ["title", "body"],
               "return_fields": ["id"],
               "filters": [("post_type", "question"),
                           ("accepted_answer_id", "*")]}
    id = outdoors_collection.search(**request)["docs"][0]["id"]
    request = {"query": id,
               "query_fields": ["id", "parent_id"],
               "limit": 3,
               "return_fields": ["id", "post_type", "title", "body", "parent_id", "accepted_answer_id"]}
    return outdoors_collection.search(**request)["docs"]

In [4]:
get_posts_for_question("climbing knots")

[{'id': '18825',
  'accepted_answer_id': 18826,
  'body': "If I wanted to learn how to tie certain knots, or learn about new knots and what they're used for, what are some good resources to look up?",
  'title': "What's a good resource for learning to tie knots for climbing?",
  'post_type': 'question'},
 {'id': '18826',
  'parent_id': 18825,
  'body': "Animated Knots By Grog Arguably the best resource online for knot tying is Animated Knots by Grog , it's used by virtually every avid knot tyer I've known. They have excellent step-by-step animations for tying hundreds of common knots for rock climbing, sailing, rescue work, fishing, etc. There are also video tutorials for each knot, and they also have an amazing mobile app that you take with you for times when you're out on an adventure and forget how to tie a knot (which I've done fishing on a number of occations).",
  'post_type': 'answer'},
 {'id': '18828',
  'parent_id': 18825,
  'body': "(In this answer I concentrate mostly on pra

## Listing 13.3

### Querying our collection with a noun phrase

In [5]:
def display_questions(query, response, verbose=False):
    print(f"Query {query}:\n")
    if verbose:
        highlights = [response["highlighting"][k] for k in response["highlighting"].keys()]
    print("Ranked Questions:")
    for i, q in enumerate(response["docs"]):
        if verbose:
            print(json.dumps(q, indent="  "))
        if "title" in q.keys():
            id = f'<a href="{q["url"]}">{q["id"]}</a>'
            display(HTML(f'<strong>Question {id}: </strong>{q["title"]}'))
        if verbose:
            display(HTML("<strong>Body:</strong>" + html.unescape(str(highlights[i]["body"][0]))))
            display(HTML("<hr>"))

def search_questions(query, verbose=False):
    request = {"query": query,
               "query_fields": ["title", "body"],
               "limit": 5,
               "return_fields": ["id", "url", "post_type", "title",
                                 "body", "accepted_answer_id", "score"],
               "filters": [("post_type", "question")],
               "order_by": [("score", "desc"), ("title", "asc")]}
    response = outdoors_collection.search(**request)
    display_questions(query, response, verbose)

In [6]:
search_questions("climbing knots")

Query climbing knots:

Ranked Questions:


## Listing 13.4

### Querying our collection with a question

In [7]:
search_questions("What is DEET?")

Query What is DEET?:

Ranked Questions:


In [8]:
search_questions("snow camping safety")

Query snow camping safety:

Ranked Questions:


Up next: [Introduction to Transformers](2.introduction-to-transformers.ipynb)